In [7]:
import io, os, sys, types
import graphlab as gl
from datetime import datetime
from os import path
import numpy as np
import import_ipynb
from decimal import *
import Conn_E_SQL as Conn
import Creacion_Vectores as C_V
import Normalizacion as Norm
from collections import OrderedDict
import operator

#Atributos de BD
host="recomdb.c4v5c6aergun.us-west-2.rds.amazonaws.com"
user="imaginex"
passwd="6^b{X2Du)Qf6"
database="RecomDB"
table= "CBF"

#Atributos de XML o consola

Type=2

#Atributos de Vivienda

Vector_BD_Vivienda= Conn.Conexion_BD(host,user,passwd,database,table)
apartment_file_processed= gl.SFrame(C_V.Creacion_Vector_Apartamento(Vector_BD_Vivienda))
apartment_file_normalized=gl.SFrame(Norm.norm_apartment_matrix(apartment_file_processed))
Vector_IDF=[]
Vector_IDF=Norm.Inv_DF(apartment_file_processed)

#Atributos de Cliente

clientfile="C:\\Users\\user\\Documents\\Sistema de Recomendacion\\Single_Client_raw.csv"
client_file_processed=gl.SFrame(C_V.Creacion_Vector_Cliente(clientfile))
client_array=client_file_processed["ID_C"]
client_number=client_array[0]
zona_array=client_file_processed["Zona"]
zona_number=zona_array[0]

#Atributos de Clientes

Vector_Clientes_BD= Conn.Conexion_BD_clients (host, user, passwd, database, table)
clients_SF_processed=gl.SFrame(C_V.Creacion_BD_Clientes(Vector_Clientes_BD))

#Atributos de Rating

Vector_Ratings_BD=Conn.Conexion_BD_rating(client_number, host, user, passwd, database, table)
client_rating=C_V.Creacion_Matrix_Rating(Vector_Ratings_BD)

#Atributos de Ratings

Vector_Ratings_BD=Conn.Conexion_BD_ratings(host, user, passwd, database, table)
clients_ratings=C_V.Creacion_Matrix_Rating(Vector_Ratings_BD)


class recommenders:
    
    def item_ranking (self, apartment_file_processed,client_number,zona_number,client_rating):
        
        #Creacion de SFrames
        
        recommender_matrix=gl.SFrame()
                
        #Visualizacion por Zona
        if (zona_number != 0):
            apartment_file_processed=apartment_file_processed[apartment_file_processed["Zona"] == zona_number]
                
        #Creacion de Modelo de Recomendacion por factorizacion
                       
        apartment_fact_recommender= gl.ranking_factorization_recommender.create(client_rating,user_id="ID_C",item_id="ID_V",target="rating",
                                                                            item_data=apartment_file_processed,
                                                                            ranking_regularization = 0.1,
                                                                            unobserved_rating_value = 1)   
                                                                            
        
              
        recommender_matrix=apartment_fact_recommender.recommend([client_number])
        
        recommender_matrix.remove_column('rank')
        
        return recommender_matrix.topk('score', k=20)
    
    def collaborative_ranking (self, apartment_file_processed, client_file_processed,clients_SF_processed,clients_ratings,visualization):
         
        print ("Ingresa la zona donde quieres ver las viviendas que compran personas parecidas a tu perfil?")
        zona=int(input())
        client_array=client_file_processed["ID_C"]
        client_number=client_array[0]
        
        #Creacion de SFrames
        
        recommender_matrix=gl.SFrame()
                    
        #Visualizacion por Zona
        if (visualization==0):
            apartment_file_processed=apartment_file_processed[apartment_file_processed["Zona"] == zona]
        
        #Creacion de Modelo de Recomendacion por Colaboracion
                       
        apartment_collaborative_recommender= gl.item_similarity_recommender.create(clients_ratings,user_id="ID_C",item_id="ID_V",target="rating",
                                                                            user_data=clients_SF_processed,
                                                                            item_data=apartment_file_processed)   
                                                                            
        
              
        recommender_matrix=apartment_collaborative_recommender.recommend([client_number])
                                                   
        recommender_matrix.remove_column('rank')
        
        return recommender_matrix.topk('score', k=20)
    
    def non_personalized_ranking (self, apartment_file_processed,clients_SF_processed,clients_ratings,client_number, zona_number):
        
                     
        #Creacion de SFrames
        
        recommender_matrix=gl.SFrame()
        clients_ratings_zone=gl.SFrame()        
        
        #Visualizacion por Zona
        if(zona_number!=0):
            apartment_file_processed=apartment_file_processed[apartment_file_processed["Zona"] == zona_number]
            zonas_ID= apartment_file_processed["ID_V"]
        else:
            zonas_ID= apartment_file_processed["ID_V"]
            
        #Modificacion de la matriz client
        
        clients_ratings_zone=clients_ratings.filter_by(zonas_ID, 'ID_V')
        
              
        #Creacion de Modelo de Recomendacion por Colaboracion
                       
        apartment_popularity_recommender= gl.recommender.popularity_recommender.create(clients_ratings_zone,user_id="ID_C",item_id="ID_V",target="rating",
                                                                            user_data=clients_SF_processed,
                                                                            item_data=apartment_file_processed)   
                                                                            
        
              
        recommender_matrix=apartment_popularity_recommender.recommend([client_number])
        recommender_matrix=recommender_matrix.remove_column("ID_C")
        recommender_matrix.remove_column('rank')
        return recommender_matrix.topk('score', k=20)
    


Finished parsing file C:\Users\user\Documents\Sistema de Recomendacion\Single_Client_raw.csv

Parsing completed. Parsed 1 lines in 0.05086 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,long,long,long,long,long,long,long,long,long,long,long,long,long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\user\Documents\Sistema de Recomendacion\Single_Client_raw.csv

Parsing completed. Parsed 1 lines in 0.064858 secs.

...

Preparing data set.

Data has 22 observations with 10 users and 381 items.

Data prepared in: 0.101728s

22 observations to process; with 381 unique items.

ID_V,score
283,5.0
211,5.0
68,5.0
44,5.0
30,5.0
4,5.0
272,4.0
250,4.0
217,4.0
139,4.0
